In [0]:
# !unzip -uq "/content/drive/My Drive/roi.zip" -d "/content/drive/My Drive/roi_sample"

In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np

import cv2
import os
import glob

Using TensorFlow backend.


In [2]:
path_plus='C:/Users/User/project_datasets/roi_set/e01' # 무표정
path_zero='C:/Users/User/project_datasets/roi_set/e02' # 웃음
path_minus='C:/Users/User/project_datasets/roi_set/e03' # 찡그림
# path_unkno='image/unknown'

la=len(os.listdir(path_zero))
lb=len(os.listdir(path_plus))
lc=len(os.listdir(path_minus))
# lc=len(os.listdir(path_unkno)) 

print('zero 경로에 저장된 파일의 개수:'+str(la))
print('plus 경로에 저장된 파일의 개수:'+str(lb))
print('minus 경로에 저장된 파일의 개수:'+str(lc))

zero 경로에 저장된 파일의 개수:500
plus 경로에 저장된 파일의 개수:500
minus 경로에 저장된 파일의 개수:500


In [3]:
xsize=200
ysize=200

In [4]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [5]:
read_dir(path_zero, 0) #'plus emotion')
read_dir(path_plus, 1) #'zero emotion')
read_dir(path_minus, 2) #'minus emotion')

In [6]:
x=np.array(x)
y=np.array(y)

In [7]:
print(x.shape)

(1500, 200, 200, 3)


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

input_shape = (xsize,ysize, 3)

batch_size = 32
num_classes = 2
epochs = 15

In [9]:
print(x_test.shape)
print(y_test.shape)

(150, 200, 200, 3)
(150,)


In [10]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.applications import DenseNet121 

input_shape = (200, 200, 3)
base_model = DenseNet121(weights=None, include_top=False, input_shape=input_shape)
x = Flatten()(base_model.output)
output = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, output=output)

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# model.summary()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 100, 100, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

__________________________________________________________________________________________________
conv3_block8_concat (Concatenat (None, 25, 25, 384)  0           conv3_block7_concat[0][0]        
                                                                 conv3_block8_2_conv[0][0]        
__________________________________________________________________________________________________
conv3_block9_0_bn (BatchNormali (None, 25, 25, 384)  1536        conv3_block8_concat[0][0]        
__________________________________________________________________________________________________
conv3_block9_0_relu (Activation (None, 25, 25, 384)  0           conv3_block9_0_bn[0][0]          
__________________________________________________________________________________________________
conv3_block9_1_conv (Conv2D)    (None, 25, 25, 128)  49152       conv3_block9_0_relu[0][0]        
__________________________________________________________________________________________________
conv3_bloc

conv4_block10_concat (Concatena (None, 12, 12, 576)  0           conv4_block9_concat[0][0]        
                                                                 conv4_block10_2_conv[0][0]       
__________________________________________________________________________________________________
conv4_block11_0_bn (BatchNormal (None, 12, 12, 576)  2304        conv4_block10_concat[0][0]       
__________________________________________________________________________________________________
conv4_block11_0_relu (Activatio (None, 12, 12, 576)  0           conv4_block11_0_bn[0][0]         
__________________________________________________________________________________________________
conv4_block11_1_conv (Conv2D)   (None, 12, 12, 128)  73728       conv4_block11_0_relu[0][0]       
__________________________________________________________________________________________________
conv4_block11_1_bn (BatchNormal (None, 12, 12, 128)  512         conv4_block11_1_conv[0][0]       
__________

conv4_block23_0_bn (BatchNormal (None, 12, 12, 960)  3840        conv4_block22_concat[0][0]       
__________________________________________________________________________________________________
conv4_block23_0_relu (Activatio (None, 12, 12, 960)  0           conv4_block23_0_bn[0][0]         
__________________________________________________________________________________________________
conv4_block23_1_conv (Conv2D)   (None, 12, 12, 128)  122880      conv4_block23_0_relu[0][0]       
__________________________________________________________________________________________________
conv4_block23_1_bn (BatchNormal (None, 12, 12, 128)  512         conv4_block23_1_conv[0][0]       
__________________________________________________________________________________________________
conv4_block23_1_relu (Activatio (None, 12, 12, 128)  0           conv4_block23_1_bn[0][0]         
__________________________________________________________________________________________________
conv4_bloc

conv5_block7_1_conv (Conv2D)    (None, 6, 6, 128)    90112       conv5_block7_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block7_1_bn (BatchNormali (None, 6, 6, 128)    512         conv5_block7_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block7_1_relu (Activation (None, 6, 6, 128)    0           conv5_block7_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_block7_2_conv (Conv2D)    (None, 6, 6, 32)     36864       conv5_block7_1_relu[0][0]        
__________________________________________________________________________________________________
conv5_block7_concat (Concatenat (None, 6, 6, 736)    0           conv5_block6_concat[0][0]        
                                                                 conv5_block7_2_conv[0][0]        
__________

flatten_1 (Flatten)             (None, 36864)        0           relu[0][0]                       
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            110595      flatten_1[0][0]                  
Total params: 7,148,099
Trainable params: 7,064,451
Non-trainable params: 83,648
__________________________________________________________________________________________________


In [ ]:
model.fit(x_train, y_train, validation_split=0.2, 
          epochs=50, batch_size=10, verbose=1)

acc = model.evaluate(x_test, y_test)
print(acc)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1080 samples, validate on 270 samples
Epoch 1/50
 100/1080 [=>............................] - ETA: 1:38:54 - loss: 10.4385 - acc: 0.2700

In [0]:
test_path = "/content/drive/My Drive/Colab Notebooks/test_set" # 기쁨만 모인 test set
test_path2 = "/content/drive/My Drive/Colab Notebooks/test_set2" # 무표정만 모인 test set 

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
          pass

In [0]:
# test set 기쁨
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
0
1
2
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1


In [0]:
# test set(무표정)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

In [0]:
# 예측(test set 기쁨)
co=0
for i in range(100):
  print(np.argmax(pred[i]),end='')
  tmp=np.argmax(pred[i])

  if tmp == 1:
    co+=1

print('\n',co/100.)


2111111111111111101111111111111111111111111212111111111111111111111211111111101201011111111111111121
 0.9


In [0]:
# 예측(test set 무표정)
co=0
for i in range(100):
  print(np.argmax(pred[i]),end='')
  tmp=np.argmax(pred[i])

  if tmp == 0:
    co+=1

print('\n',co/100.)

1101110112110111110111201121111111111111011200011111122100100111021001101111011011210122112111121111
 0.2
